In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 4)
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
from scipy.io.wavfile import read
import librosa
from librosa import feature, frames_to_time, autocorrelate, clicks

## Tempo Estimation

The beat (or pulse) of a musical piece is a *periodic* sequence of events or impulses. By examining the periodic structure of the impluses, we can try to estimate what one period is (the distance of a beat), which will allow us to guess the tempo.

### Autocorrelation

We have learned about correlation. Autocorrelation is used to find how similar a signal, or function, is *to itself* at a certain time difference (or shifted position), which is referred to as *lag*.

For example, let's say I have the following sequence:

In [ ]:
vals = np.array([4,1,3,1,4,1.5,2.5,0.2,3.5,1,2,1.5])

Let's make a plot to see it better:

In [ ]:
plt.stem(vals)

An autocorrelation will compare this original array to every possible shifted version of itself.

In [ ]:
a = np.roll(vals, -1)
b = np.roll(vals, -2)
c = np.roll(vals, -3)
#d = np.roll(vals, -4)

plt.stem(vals)
(markers, stemlines, baseline) = plt.stem(b)
plt.setp(markers, marker='D', markersize=10, markeredgecolor="orange", markeredgewidth=2)
plt.setp(stemlines, color='orange')


In [ ]:
np.corrcoef(vals, c)

## Test it out on Energy values

#### Recall from our last lesson that to estimate onsets, the steps are: 
1. calculate the energy or RMSE


In [ ]:
from scipy.io.wavfile import read
(fs, x) = read('../audio/80spopDrums.wav')
xnorm = x/np.abs(x.max())

In [ ]:
time = np.arange(0,xnorm.size/fs,1/fs)

hop_length = 512 # 50% overlap
frame_length = 1024

In [ ]:
from librosa import feature, frames_to_time
rms = feature.rms(y=xnorm, hop_length=hop_length, frame_length=frame_length)

In [ ]:
len(rms[0])

In [ ]:
frames = range(len(rms[0]))
t = frames_to_time(frames, sr=fs, hop_length=hop_length)

In [ ]:
t.shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,4))

plt.plot(time, xnorm, alpha=0.3)
plt.plot(t, rms[0], 'g--');

2. calculate the frame-to-frame difference in energy. This helps highlight moments of increasing energy—likely places where musical onsets (like drum hits) occur.


In [ ]:
diff = rms[0][1:] - rms[0][0:-1]
#half-wave rectification - use numpy.where to set elements of array when condition is satisfied

In [ ]:
plt.plot(diff)

3. We then apply half-wave rectification (i.e., zeroing out negative values), so only energy increases remain, and decreases have been "thrown out"

In [ ]:
diff_hw = np.where(diff < 0, 0, diff)
# diff[diff < 0 ] = 0 # alternative method

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(t[1:], diff_hw);# drop first time index so that arrays match & have proper start/shift


4. Locate the frames that pass the threshold

In [ ]:
# create arbitrary cutoff (try testing it out)
# you may want to try normalizing these difference values!

xplines = diff > 0.03
xplines[:20]


Here we can visualize what we've done so far by plotting our signal underneath vertical lines representing our "guesses" at the onset positions.

In [ ]:
plt.figure(figsize=(15,4))

plt.plot(time, xnorm, alpha=0.3)
plt.plot(t, rms[0], 'g--');

# use the boolean mask to locate the frame values exceeding the threshold
for xc in t[1:][xplines]:
    plt.axvline(x=xc, linestyle='--')

We can use `librosa.clicks` to check how we are doing (https://librosa.org/doc/0.8.0/generated/librosa.clicks.html)by passing our time array of x-axis 'lines' (onset guesses) to convert them to pulses (clicks).


In [ ]:
# clicks is a librosa function
clicks = librosa.clicks(times=t[1:][xplines], sr=fs, hop_length=None)

In [ ]:
print(xnorm.size, clicks.size)

In [ ]:
clicks.resize(xnorm.shape)
combn = clicks + xnorm

In [ ]:
from IPython.display import Audio

In [ ]:
Audio(combn, rate=fs)

### Here's where autocorrelation comes in...

Let's apply autocorrelation to our energy (measured as RMSE) differential, to look for periodicities in the distance between spikes.

Whereas `np.roll()` uses a rotating vector (i.e., takes the values from the end and places them at the beginning) in audio signal autocorrelation we don't want to reorganize the signal values, so the method instead is to remove the values that you have shifted over, and replace them with zeros.


In [ ]:
ac = librosa.autocorrelate(diff_hw) # check defaults

### How does `librosa.autocorrelate()` work?

When you use `librosa.autocorrelate(x)`, it computes the **non-circular autocorrelation** of the signal. Under the hood, it behaves similarly to:

`np.correlate(x, x, mode='full')[len(x)-1:]`

#### Mathematical Definition
The autocorrelation at lag $\tau$ is defined as:
$$
R(\tau) = \sum_{t=0}^{N - \tau - 1} x[t] \cdot x[t + \tau]
$$

Where:

$x[t]$ is the signal value at time $t$

$N$ is the total number of samples

$\tau$ is the lag (in samples)

This gives us a measure of how similar the signal is to itself when shifted by $\tau$ steps.

Notice this is **exactly** the same as our DFT process, except we are multiplying and summing not with a basis function (sinusoid) but a shifted version of the input itself.

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(ac)
plt.title('Autocorrelation of Energy differential')
plt.xlabel('Time lag (in frames)')

In [ ]:
ac.max()

In [ ]:
ac[0]

In [ ]:
v = ac[1:].max()
v

In [ ]:
points = np.where(ac > 0.4) # ignore zero and one indices
points

Calculate the times between successive spikes (each frame increments by 512 samples).

In [ ]:
fsec = 512/fs
fsec

In [ ]:
points = np.array(points)
t_inc = fsec * points
#ignore first two values (zero and one)
t_inc[0][2:]

In [ ]:
#calculate the differences between the values
ts = t_inc[0][2:]

durs = ts[1:] - ts[:-1]
durs = np.round(durs, 3) # round to 3 decimal places
durs

In [ ]:
from collections import Counter

# Get the top N most common elements, let's say top 3 for this example
num_counts = Counter(durs)
top_n = num_counts.most_common(3)

print(top_n)


Notice that .011, .276, and .534 (approximately) recur (in seconds). 60 divided by these values gets us estimated tempo.

I.e., 

The tempo in beats per minute (BPM) is calculated as:

$$
\text{Tempo (BPM)} = \frac{60}{\text{Seconds per Beat}}
$$

So if the periodicity we detect is approximately 0.534 seconds:

$$
\frac{60}{0.534} \approx 112.36 \text{ BPM}
$$

In [ ]:
tempa = 60 / .012
tempb = 60 / .267
tempc = 60 / .534

print( tempa, tempb, tempc)

Given the "normal" range of tempi, tempc is the most likely. Check with tempo calculator.


### Other Uses of Autocorrelation in Music DSP

- **Pitch Detection**: Repeating waveforms (e.g., from sung vowels) can be tracked using autocorrelation to estimate pitch.
- **Meter Estimation**: Beyond tempo, we can use autocorrelation hierarchically to estimate larger structures, such as meters. This involves looking for subharmonics or harmonics in the autocorrelation peaks.
- **Loop Point Detection**: In audio looping (like in sampling or sound design), autocorrelation can help find natural looping points by identifying repeated signal patterns.
- **Periodicity-Based Segmentation**: Some rhythm or style classifiers use autocorrelation to segment time-series signals based on repeating patterns.


## Meter Estimation
With meter, our beats recur in patterns of stronger and weaker beats. This hierarchical structure can be detected by examining **the periodicity of the signal at multiple levels.**

#### Hierarchical Autocorrelation for Meter
To estimate meter, we can apply autocorrelation not just at the level of the beat (as we did for tempo) but also at sub-beats and higher-order periodicities (e.g., the measure). This involves looking for autocorrelation peaks at different time scales:

**Beat level:** The fundamental period we detected earlier with autocorrelation corresponds to the basic tempo of the piece.

**Sub-beat level:** To detect subdivisions of the beat (like eighth notes or sixteenth notes), we look for peaks at half or quarter the beat's period.

**Measure level:** We can further examine higher-order periodicities corresponding to measures, typically involving periods that are multiple times the beat.

For example: after detecting the beat period, we can check for harmonics of that period (i.e., periods that are integer multiples of the beat period) in the autocorrelation result. The presence of significant peaks at these multiple timescales indicates a strong meter structure.